In [1]:
import numpy as np
import typing as tp
import math
import inspect
def check(bruh):
    print("Is this a class name?... "+str(inspect.isclass(bruh)))
class NeuralNetwork:
    pass

In [106]:
class LossFunction: 
    pass 
class MSE: # Includes Regularization, Mean Squared Error
    def __init__(self, Lambda):
        self._Lambda = Lambda # Lambda for L2 regularization
        self.__LastOutput = np.zeros((1,1))
        
    def f(self, NNET: NeuralNetwork, Output: np.array, Truth: np.array):
        L2 = 0
        for W in NNET._WEIGHTS:
            L2 += np.sum(W ** 2)            
        return 0.5/Output.size * (np.sum((Output - Truth) ** 2) + self._Lambda * L2) # Watch out for Backprop and all that for L2
    
    def df(self, NNET: NeuralNetwork, Output: np.array, Truth: np.array) -> np.array:
        return 1/Output.size * ((Output - Truth))
    
class ActivationFunction:
    pass
class RELU(ActivationFunction): # RELU activation, for other activations last output will be saved for faster derivative
    def __init__(self):
        self.__LastOutput = np.zeros((1,1))
        self.f = np.vectorize(self.__f)
        self.df = np.vectorize(self.__df)
        
    def __f(self, Input):
        if Input > 0:
            return Input
        else:
            return 0
        
    def __df(self, Input): 
        if Input > 0: 
            return 1
        else:
            return 0
        pass
    
    
class Optimizer:
    pass
class OptimizerExample(Optimizer):
    def __init__(self, Alpha, Beta): 
        self.__Time = 0 # For decays and that, incremented each epoch maybe
        self.__Alpha = Alpha # For momentum
        self.__Beta = Beta # For momentum
        self.__Vw = [] # Update gradient for weights for nesterov Momentum
        self.__Vb = [] # Update gradient for biases
        self.__LastOutput = [] # Last output of each layer

        self.__VPass = []
        self.__VTemp = []
        # ... Missing some other varibles for backpropagation
        
    def Optimize(self, NNET: NeuralNetwork, Input: np.array, Truth: np.array ) -> tp.Tuple[tp.List[np.array], tp.List[np.array]]:    
        N = len(NNET._WEIGHTS)
        InvOutputSize = 1/Truth[0].size
        Loss = 0

        self.__Vw = [None] * N # Initialize variables to mantain consistance across epochs
        self.__Vb = [None] * N
        self.__LastOutput = [None] * N

        self.__VPass = [None] * N
        self.__VTemp = [None] * N

    
        for Temp in range(0, N):
            self.__Vw[Temp] = np.zeros(NNET._WEIGHTS[Temp].shape)  # Initialize prev Vw to 0's
        
        for Elem in range(0, Input.shape[0]): # For every element in training input batch
            # Forward Propagation
            self.__LastOutput[0] = NNET._ACTIVATION[0].f(np.matmul(NNET._WEIGHTS[0], Input[Elem]) + NNET._BIASES[0]) # First we need the input to propagate
            for i in range(1, N):
                self.__LastOutput[i] = NNET._ACTIVATION[i].f(np.matmul(NNET._WEIGHTS[i], self.__LastOutput[i-1]) + NNET._BIASES[i]) # Propagate forward and save outputs
            # We get the error outside loop
            Error = NNET._LOSS.f(NNET, self.__LastOutput[N-1], Truth[Elem])


           # print(NNET._WEIGHTS[N-1])
            print("Loss: "+ str(Error))


            # Backpropagation
            self.__VPass[N-1] = NNET._LOSS.df(NNET, self.__LastOutput[N-1], Truth[Elem]) # First propagation passed back is the derivative of the error (with respect to the output)
            for i in range(N-1, 0, -1):
                self.__VTemp[i] = np.multiply(self.__VPass[i], NNET._ACTIVATION[i].df(self.__LastOutput[i])) # Useful Temp array (which is in reality dE/db, maybe later will be deprecated to optimize)
                self.__Vw[i] = self.__Alpha * np.outer(self.__VTemp[i], self.__LastOutput[i-1]) + self.__Beta * self.__Vw[i] +  NNET._LOSS._Lambda * InvOutputSize * NNET._WEIGHTS[i] # Weights gradient, includes L2 propagated. This will not be possible for firs layer, so we need outside loop function (we could include the input into the output list, but idk)
                self.__Vb[i] = self.__Alpha * self.__VTemp[i] # Biases gradient
                self.__VPass[i-1] = self.__VTemp[i].dot(NNET._WEIGHTS[i]) # Propagated error
               # print("This should be ("+str(self.__VTemp[i].size)+", "+str(self.__LastOutput[i-1].size)+"): " + str(self.__Vw[i].shape))
                NNET._WEIGHTS[i] -= self.__Vw[i] # Update parameters
                NNET._BIASES[i] -= self.__Vb[i]
            # First layer we do outside loop
            self.__VTemp[0] = np.multiply(self.__VPass[0], NNET._ACTIVATION[0].df(self.__LastOutput[0])) 
            self.__Vw[0] = self.__Alpha * np.outer(self.__VTemp[0], Input[Elem]) + self.__Beta * self.__Vw[0] +  InvOutputSize * NNET._WEIGHTS[0] 
            self.__Vb[0] = self.__Alpha * self.__VTemp[0] # Biases gradient 
            NNET._WEIGHTS[0] -= self.__Vw[0] # Update parameters
            NNET._BIASES[0] -= self.__Vb[0]

        return NNET._WEIGHTS, NNET._BIASES

    
class NeuralNetwork:
    def __init__(self, ActivationFunctions: tp.List[ActivationFunction], Nodes: tp.List[int]):        
        self._ACTIVATION = ActivationFunctions # List with activation function, objects
        self._LOSS = MSE(0.001) # Loss function, object
        self._WEIGHTS = [] # Weight list 
        self._BIASES = [] # Biases list 
        
        # Add: 
        # Variable input and ouput sizes
        # Different activation functions ~ start with ELU
        # L1 regularization
        # Nesterov SGD with momentum optimizer
        # Initializer objects
        # Dropout
        
        for i in range(1, len(Nodes)): # Initialize weights and biases sizes
            self._WEIGHTS.append(np.ones((Nodes[i], Nodes[i-1])))
            self._BIASES.append(np.ones(Nodes[i]))
            
    def Train(self, Input: np.array, Truth: np.array, Iterations: int, Optimizer_: Optimizer, LossFunction_: LossFunction = None ) -> None: # Truth.shape = (TrainingInstances, ElementsInEach), same for input     
 
        if LossFunction_ is not None: # If other loss function is provided
            self._LOSS = LossFunction_
            
        for Iter in np.arange(0, Iterations):
                self._WEIGHTS, self._BIASES = Optimizer_.Optimize(self, Input, Truth) # Optimizer updates biases and weights each epoch
                
    
            
    

In [110]:
N = NeuralNetwork([RELU(), RELU(), RELU()], [5,8,8,2]) # Pass objects!!1
N.Train(np.random.rand(1,5), np.random.rand(1,2), 100000, OptimizerExample(0.001,0.001))

Se han truncado las últimas 5000 líneas del flujo de salida.
Loss: 0.0006819172736256846
Loss: 0.0006818321728541501
Loss: 0.0006817470832421713
Loss: 0.0006816620047881521
Loss: 0.0006815769374904954
Loss: 0.000681491881347602
Loss: 0.0006814068363578743
Loss: 0.0006813218025197161
Loss: 0.0006812367798315276
Loss: 0.0006811517682917138
Loss: 0.0006810667678986776
Loss: 0.0006809817786508207
Loss: 0.0006808968005465495
Loss: 0.0006808118335842631
Loss: 0.0006807268777623737
Loss: 0.0006806419330792797
Loss: 0.0006805569995333864
Loss: 0.0006804720771231033
Loss: 0.0006803871658468312
Loss: 0.0006803022657029765
Loss: 0.0006802173766899443
Loss: 0.0006801324988061451
Loss: 0.0006800476320499825
Loss: 0.0006799627764198633
Loss: 0.0006798779319141914
Loss: 0.0006797930985313783
Loss: 0.0006797082762698313
Loss: 0.0006796234651279561
Loss: 0.0006795386651041604
Loss: 0.000679453876196854
Loss: 0.000679369098404445
Loss: 0.0006792843317253425
Loss: 0.0006791995761579544
Loss: 0.0006791148

KeyboardInterrupt: ignored

In [ ]:
Çfrom IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def notV(x):
    if x<0:
        return x
    else:
        return 1-math.exp(-x)

In [ ]:
V = np.vectorize(notV)

In [ ]:
Y = np.random.rand(10,1)
X = V(Y)

In [ ]:
def testS(x):
    print(np.matmul(np.ones((10,3)), x[0]))

In [ ]:
C = np.random.rand(6,3)

In [ ]:
def temp(x):
    print(x.size)

In [67]:
X = np.random.randrange((3,8))
Y = np.asarray([2,3,4])

Y.dot(X)

AttributeError: ignored

temp(C)